In [ ]:
%tensorflow_version 2.x

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import sys
import os
sys.path.append('/content/gdrive/My Drive/W266-NLP/Project')
os.chdir('/content/gdrive/My Drive/W266-NLP/Project')
os.listdir()

In [1]:
import os
import numpy as np
import pickle
import pandas as pd
import json
from pandas.io.json import json_normalize
from collections import Counter
import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Attention
import nltk
from functools import reduce
!pip install wget
# Load PyDrive and Google Auth related packages
#!pip install -U -q PyDrive
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Attention
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input
from functools import reduce
flatten = lambda data: reduce(lambda x, y: x + y, data)
import glove_helper

# Load the json data
def load_json_file(name):
  """
  Load the json file and return a json object
  """
  with open(name,encoding='utf-8') as myfile:
    data = json.load(myfile)
    return data

# Convert json data object to a pandas data frame
def convert_to_pd(data):
  """
  Load the data to a pandas dataframe.
  Dataframe Columns:
    title
    para_index
    context
    q_index
    q_id
    q_isimpossible
    q_question
    q_anscount - number of answers
    q_answers - a list of object e.g [{ text: '', answer_start: 123}, ...]
  """
  result = []
  for pdata in data['data']:
    for para in pdata['paragraphs']:
      for q in para['qas']:
        result.append({
            'title' : pdata['title'],
            'context' : para['context'],
            'q_id' : q['id'],
            'q_isimpossible' : q['is_impossible'],
            'q_question' : q['question'],
            'q_anscount' : len(q['answers']),
            'q_answers' : [a for a in q['answers']],
            'q_answers_text': [a.get("text") for a in q['answers']],
            'context_lowercase': para['context'].lower(),
            'q_question_lowercase' : q['question'].lower(),
            'q_answers_text_lowercase': [a.get("text").lower() for a in q['answers']],
            
        })

  return pd.DataFrame.from_dict(result, orient='columns')

# Load the file from shareable google drive link and return a pandas dataframe
def loadDataFile(filename): 
  """
  Download a file from google drive with the shared link
  """ 
  data = load_json_file(filename)
  return convert_to_pd(data)

In [2]:
# DONOT RUN THIS ON COLAB#
#to make use of CPU and not GPU DONOT RUN THIS ON COLAB
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""


In [2]:
train_filename = 'train-v2.0.json'
dev_filename = 'dev-v2.0.json'

train_pd = loadDataFile(train_filename)
dev_pd = loadDataFile(dev_filename)

In [3]:
def get_c_q_a(dataset):
    q_id_list = []
    context_list =[]
    questions_list = []
    answers_list =[]
    q_impossible_list =[]
    for index,row in dataset.iterrows():
        q_id_list.append(row.q_id)
        context_list.append(row.context)
        questions_list.append(row.q_question)
        q_impossible_list.append(int(row.q_isimpossible))
        if len(row.q_answers_text)>0 :
            answers_list.append(row.q_answers_text[0])
        else:
            answers_list.append("")
    return [q_id_list,context_list,questions_list,q_impossible_list,answers_list]

train_lists = get_c_q_a(train_pd)
dev_lists = get_c_q_a(dev_pd)
context_maxlen = max(map(len, (x.split() for x in train_lists[1])))
question_maxlen = max(map(len, (x.split() for x in train_lists[2])))
answer_maxlen = max(map(len, (x.split() for x in train_lists[4])))
print("Max context length:",context_maxlen)
print("Max question length:",question_maxlen)
print("Max answer length:",answer_maxlen)

Max context length: 653
Max question length: 40
Max answer length: 43


In [4]:
context_maxlen = 214
question_maxlen = 18
answer_maxlen = 10

In [5]:
def tokenize_c_q_a(dataset,num_words=None):
    tokenizer = Tokenizer(num_words,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"''",oov_token='<unk>')
    data = dataset[1]+dataset[2]+dataset[4]
    tokenizer.fit_on_texts(data)
    vocab = {}
    for word,i in tokenizer.word_index.items():
        if num_words is not None:
          if i <= num_words:
            vocab[word] = i
        else:
          vocab[word] = i
    #vocab = tokenizer.word_index
    vocab['<s>'] = len(vocab)+1
    vocab['</s>'] = len(vocab)+1
    id_vocab = {value: key for key, value in vocab.items()}
    return (tokenizer,vocab,id_vocab)

tokenizer_obj,vocab,id_vocab = tokenize_c_q_a(train_lists)

vocab_size = len(vocab)
print("Vocab Size:",vocab_size)

def vectorize_data(tokenizer_obj,train_lists):
    context_seq = tokenizer_obj.texts_to_sequences(train_lists[1])
    question_seq = tokenizer_obj.texts_to_sequences(train_lists[2])
    answer_seq = tokenizer_obj.texts_to_sequences(train_lists[4])
    answer_input_seq = [[vocab['<s>']]+i+[vocab['</s>']] for i in answer_seq]
    answer_target_seq = [i+[vocab['</s>']] for i in answer_seq]
    context_seq_padded = pad_sequences(context_seq,context_maxlen,padding='post', truncating='post')
    question_seq_padded = pad_sequences(question_seq,question_maxlen,padding='post', truncating='post')
    answer_seq_padded = pad_sequences(answer_seq,answer_maxlen,padding='post', truncating='post')
    answer_input_seq_padded = pad_sequences(answer_input_seq,answer_maxlen+2,padding='post', truncating='post')
    answer_target_seq_padded = pad_sequences(answer_target_seq,answer_maxlen+2,padding='post', truncating='post')
    answer_impossible = np.array(train_lists[3])
    indices = np.arange(context_seq_padded.shape[0])
    np.random.seed(19)
    np.random.shuffle(indices)
    context_seq_padded = context_seq_padded[indices]
    question_seq_padded = question_seq_padded[indices]
    answer_seq_padded = answer_seq_padded[indices]
    answer_input_seq_padded = answer_input_seq_padded[indices]
    answer_target_seq_padded = answer_target_seq_padded[indices]
    answer_impossible_shuffled = answer_impossible[indices]
    train_samples = int(((context_seq_padded.shape[0]*.8)//128)*128)
    end_samples = int((context_seq_padded.shape[0]//128)*128)
    train_context_padded_seq = context_seq_padded[:train_samples]
    train_question_seq_padded = question_seq_padded[:train_samples]
    train_answer_seq_padded = answer_seq_padded[:train_samples]
    train_answer_input_seq_padded = answer_input_seq_padded[:train_samples]
    train_answer_target_seq_padded = answer_target_seq_padded[:train_samples]
    train_answer_impossible = answer_impossible_shuffled[:train_samples]
    val_context_padded_seq = context_seq_padded[train_samples:end_samples]
    val_question_seq_padded = question_seq_padded[train_samples:end_samples]
    val_answer_seq_padded = answer_seq_padded[train_samples:end_samples]
    val_answer_input_seq_padded = answer_input_seq_padded[train_samples:end_samples]
    val_answer_target_seq_padded = answer_target_seq_padded[train_samples:end_samples]
    val_answer_impossible = answer_impossible_shuffled[train_samples:end_samples]
    return (train_context_padded_seq,train_question_seq_padded,train_answer_seq_padded,
            train_answer_input_seq_padded,train_answer_target_seq_padded,train_answer_impossible,
            val_context_padded_seq,val_question_seq_padded,val_answer_seq_padded,
            val_answer_input_seq_padded,val_answer_target_seq_padded,val_answer_impossible)

train_context_padded_seq,train_question_seq_padded,train_answer_seq_padded,\
train_answer_input_seq_padded,train_answer_target_seq_padded,\
train_answer_impossible,\
val_context_padded_seq,val_question_seq_padded,val_answer_seq_padded,\
val_answer_input_seq_padded,val_answer_target_seq_padded,\
val_answer_impossible\
= vectorize_data(tokenizer_obj,train_lists)

print("validation num samples where answer impossible: ",len(val_answer_seq_padded[val_answer_impossible==1]))
print("validation num samples where answer not impossible: ",len(val_answer_seq_padded[val_answer_impossible==0]))
print("train num samples where answer impossible: ",len(train_answer_seq_padded[train_answer_impossible==1]))
print("train num samples where answer not impossible: ",len(train_answer_seq_padded[train_answer_impossible==0]))

Vocab Size: 88701
validation num samples where answer impossible:  8730
validation num samples where answer not impossible:  17382
train num samples where answer impossible:  34761
train num samples where answer not impossible:  69431


In [6]:
def create_embedding_matrix(word_index,vocab_size=50000,ndim=100):
    hands = glove_helper.Hands(ndim)
    embedding_matrix = np.zeros((vocab_size+1,ndim))
    for word,i in word_index.items():
        if i<=vocab_size:
            embedding_vector = hands.get_vector(word,strict=False)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix
ndim = 100
embedding_matrix = create_embedding_matrix(vocab,vocab_size,ndim)

Loading vectors from data/glove/glove.6B.zip
Parsing file: data/glove/glove.6B.zip:glove.6B.100d.txt
Found 400,000 words.
Parsing vectors... Done! (W.shape = (400003, 100))


In [22]:
#Function to create the Models
def create_models(embedding_matrix,
                  max_context_length,
                  max_question_length,
                  max_answer_length,
                  num_unit_gru = 64,
                  num_layers_gru = 2,
                  ndim =100,
                  num_episodes = 2,
                  num_dense_layer_feasibility_units = 16,
                  dropout_rate = 0.5,
                  num_dense_layers_feasibility = 1,
                  num_episodic_network_unit = 64):
    
    def create_episodic_memory(num_episodes,
                               query,
                               context_outputs,
                               max_context_length,
                               max_question_length,
                               num_episodic_network_unit):
        m = layers.Lambda(lambda x: x)(query)
        weight_layer = layers.Dense(query.shape[1],use_bias=False)
        for i in range(num_episodes):
            m_increased = tf.tile(tf.keras.backend.expand_dims(m,1),
                                  tf.constant([1,max_context_length,1],tf.int32))
            q_increased = tf.tile(tf.keras.backend.expand_dims(query,1),
                                  tf.constant([1,max_context_length,1],tf.int32))
            c_mul_q = layers.multiply([context_outputs,q_increased])
            c_mul_m = layers.multiply([context_outputs,m_increased])
            c_minus_q =tf.keras.backend.abs(layers.subtract([context_outputs,q_increased]))
            c_minus_m = tf.keras.backend.abs(layers.subtract([context_outputs,m_increased]))
            c_dot_q = tf.matmul(tf.keras.backend.expand_dims(weight_layer(query),1), context_outputs,transpose_b=True)
            c_dot_q = layers.Permute((2,1))(c_dot_q)
            c_dot_m = tf.matmul(tf.keras.backend.expand_dims(weight_layer(m),1), context_outputs,transpose_b=True)
            c_dot_m = layers.Permute((2,1))(c_dot_m)
            z = layers.concatenate([context_outputs,
                                    m_increased,
                                    q_increased,
                                    c_mul_q,
                                    c_mul_m,
                                    c_minus_q,
                                    c_minus_m,
                                    c_dot_q,
                                    c_dot_m],axis=-1)
            score = layers.Dense(1)(layers.Dense(num_episodic_network_unit,activation='tanh')(z))
            attention_weights = tf.nn.softmax(score, axis=1)
            m_value = attention_weights * context_outputs
            m = tf.reduce_sum(m_value, axis=1)
        return m
    
    
    #Input Module
    context_input = Input(shape=(None,),dtype='int32',name='Context_Input')
    context_embeddings = layers.Embedding(vocab_size+1,
                                          ndim,
                                          mask_zero=True,
                                          name='Context_Embedding')(context_input)

    for i in range(num_layers_gru):
        context_outputs_layers = layers.Bidirectional(layers.GRU(num_unit_gru,
                                                                 dropout=dropout_rate,
                                                                 recurrent_dropout= dropout_rate,
                                                                 recurrent_initializer='glorot_uniform',
                                                                 return_sequences=True),
                                                      merge_mode='sum',
                                                      name='Context_Bid_Layer'+str(i))
        if i==0:
            context_outputs = context_outputs_layers(context_embeddings)
        else:
            context_outputs = context_outputs_layers(context_outputs)
        context_outputs = layers.BatchNormalization()(context_outputs)
    print("Context output shape",context_outputs.shape)
    #Question Module
    question_input = Input(shape=(None,),dtype='int32',name='Question_Input')
    question_embeddings = layers.Embedding(vocab_size+1,
                                           ndim,
                                           mask_zero=True,
                                           name='Question_Embedding')(question_input)

    for i in range(num_layers_gru):
        if i==0 and num_layers_gru >1:
            question_outputs = layers.Bidirectional(layers.GRU(num_unit_gru,
                                                               dropout=dropout_rate,
                                                               recurrent_dropout= dropout_rate,
                                                               recurrent_initializer='glorot_uniform',
                                                               return_sequences=True),
                                                    merge_mode='sum',
                                                    name='Question_Bid_Layer'+str(i))(question_embeddings)
        elif i==0 and num_layers_gru ==1:
            question_outputs = layers.Bidirectional(layers.GRU(num_unit_gru,
                                                               dropout=dropout_rate,
                                                               recurrent_dropout= dropout_rate,
                                                               recurrent_initializer='glorot_uniform',
                                                               return_sequences=False),
                                                    merge_mode='sum',
                                                    name='Question_Bid_Layer'+str(i))(question_embeddings)
        elif i==(num_layers_gru-1):
            question_outputs = layers.Bidirectional(layers.GRU(num_unit_gru,
                                                               dropout=dropout_rate,
                                                               recurrent_dropout= dropout_rate,
                                                               recurrent_initializer='glorot_uniform',
                                                               return_sequences=False),
                                                    merge_mode='sum',
                                                    name='Question_Bid_Layer'+str(i))(question_outputs)
        else:
            question_outputs = layers.Bidirectional(layers.GRU(num_unit_gru,
                                                               dropout=dropout_rate,
                                                               recurrent_dropout= dropout_rate,
                                                               recurrent_initializer='glorot_uniform',
                                                               return_sequences=True),
                                                    merge_mode='sum',
                                                    name='Question_Bid_Layer'+str(i))(question_outputs)
        question_outputs = layers.BatchNormalization()(question_outputs)
    #Episodic Memory Module
    m=create_episodic_memory(num_episodes,
                             question_outputs,
                             context_outputs,
                             max_context_length,
                             max_question_length,
                             num_episodic_network_unit)
    #print(m.shape)
    #print(context_outputs.shape)
    #print(question_outputs.shape)
    concatenated_tensor = layers.concatenate(inputs=[m,question_outputs],
                                             name='Concatenation_Memory_Question',axis=1)
    #answer_module

    answer_input = Input(shape=(None,),dtype='int32',name='Answer_Input')
    answer_embeddings = layers.Embedding(vocab_size+1,
                                         ndim,
                                         mask_zero=True,
                                         name='Answer_Embedding')(answer_input)
    answer_embedding_plus_question = layers.concatenate(
                                       inputs = [tf.tile(tf.keras.backend.expand_dims(question_outputs, 1),
                                                         tf.constant([1,max_answer_length+2,1],
                                                                    tf.int32)),
                                                 answer_embeddings],axis=-1)
    for i in range(num_layers_gru):
        answer_decoder_layers = layers.GRU(num_unit_gru,
                                           dropout=dropout_rate,
                                           recurrent_dropout= dropout_rate,
                                           recurrent_initializer='glorot_uniform',
                                           return_sequences=True,
                                           return_state=True,
                                           name='Answer_GRU_Layer'+str(i)
                                           )
        if i==0:
            answer_outputs,_ = answer_decoder_layers(answer_embedding_plus_question,initial_state=m)
        else:
            answer_outputs,_ = answer_decoder_layers(answer_outputs)
        answer_outputs = layers.BatchNormalization()(answer_outputs)
    answer_decoder_dense = layers.TimeDistributed(layers.Dense(vocab_size+1, activation='softmax')
                                                  ,name='Answer_output')
    answer_decoder_outputs = answer_decoder_dense(answer_outputs)

    answer_model = Model([context_input,question_input,answer_input],answer_decoder_outputs)
    answer_model.get_layer("Question_Embedding").set_weights([embedding_matrix])
    answer_model.get_layer("Question_Embedding").trainable = False
    answer_model.get_layer("Context_Embedding").set_weights([embedding_matrix])
    answer_model.get_layer("Context_Embedding").trainable = False
    answer_model.get_layer("Answer_Embedding").set_weights([embedding_matrix])
    answer_model.get_layer("Answer_Embedding").trainable = False
    
    #feasibility module
    feasibility_input = Input(shape=(concatenated_tensor.shape[1],), name="FeasibilityInput")
    for i in range(num_dense_layers_feasibility):
        if i==0:
            dense_layer = layers.Dense(num_dense_layer_feasibility_units,
                            activation='relu',name='feasibility_layer_'+str(i))(feasibility_input)
        else:
            dense_layer = layers.Dense(num_dense_layer_feasibility_units,
                            activation='relu',name='feasibility_layer_'+str(i))(dense_layer)
        dense_layer = layers.BatchNormalization()(dense_layer)
        dropout_layer = layers.Dropout(dropout_rate,name='feasibility_drop_'+str(i))(dense_layer)

    feasibility_output = layers.Dense(1,activation='sigmoid',name='feasibility_output')(dropout_layer)
    feasibility_model = Model(feasibility_input,feasibility_output)

    encoder_model = Model([context_input,question_input], [m,concatenated_tensor,question_outputs])
    decoder_inputs = answer_input
    decoder_state_input_h = Input(shape=(None,), name="DecoderStateInput")
    decoder_question_input = Input(shape=(None,),name="DecoderQuestionInput")
    decoder_answer_embeddings = answer_model.get_layer('Answer_Embedding')(decoder_inputs)
    decoder_answer_embedding_plus_question = layers.concatenate(
                                                 inputs = [tf.keras.backend.expand_dims(decoder_question_input, 1),
                                                 decoder_answer_embeddings],axis=-1)

    for i in range(num_layers_gru):
        decoder_layers = answer_model.get_layer('Answer_GRU_Layer'+str(i))
        if i==0:
            decoder_outputs, decoder_state_h = decoder_layers(decoder_answer_embedding_plus_question,
                                                              initial_state=decoder_state_input_h)
        else:
            decoder_outputs, decoder_state_h = decoder_layers(decoder_outputs)

    decoder_dense =  answer_model.get_layer('Answer_output')(decoder_outputs)

    decoder_model = Model(
                        [decoder_inputs] + [decoder_state_input_h]+[decoder_question_input],
                        [decoder_dense] + [decoder_state_h])
    return (answer_model,encoder_model,decoder_model,feasibility_model)

In [23]:
#Function to get sentences from the predicted answers
def decode_sequence(context_input_seq,
                    question_input_seq,
                    encoder_model,
                    decoder_model):
    # Encode the input as state vectors.
    states_value,_,question_outputs = encoder_model.predict([context_input_seq,question_input_seq])

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    current_step = 0
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = vocab["<s>"]

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h = decoder_model.predict([target_seq] + [states_value] + [question_outputs])
        current_step += 1
        # Sample a token
        #print(output_tokens.shape)
        #print(output_tokens[0,0,0])
        #print(output_tokens[0,0,32984])
        sampled_token_index = np.argmax(output_tokens[0, 0, :])
        #print(sampled_token_index)
        if sampled_token_index == 0:
            sampled_char = " "
        else:
            sampled_char = id_vocab[sampled_token_index]
        decoded_sentence += sampled_char + " "

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == '</s>' or len(decoded_sentence) > answer_maxlen:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = h

    return decoded_sentence

In [24]:
Experiment_Dic = {'Experiment0': {'num_unit_gru': 64, 'num_layers_gru': 2, 'num_episodes': 3, 'num_dense_layer_feasibility_units': 32, 'dropout_rate': 0.6, 'num_dense_layers_feasibility': 1, 'num_episodic_network_unit': 192, 'learning_rate': 0.001}, 'Experiment1': {'num_unit_gru': 80, 'num_layers_gru': 2, 'num_episodes': 2, 'num_dense_layer_feasibility_units': 64, 'dropout_rate': 0.7, 'num_dense_layers_feasibility': 3, 'num_episodic_network_unit': 192, 'learning_rate': 0.005}, 'Experiment2': {'num_unit_gru': 100, 'num_layers_gru': 2, 'num_episodes': 3, 'num_dense_layer_feasibility_units': 64, 'dropout_rate': 0.5, 'num_dense_layers_feasibility': 1, 'num_episodic_network_unit': 64, 'learning_rate': 0.005}, 'Experiment3': {'num_unit_gru': 80, 'num_layers_gru': 2, 'num_episodes': 2, 'num_dense_layer_feasibility_units': 32, 'dropout_rate': 0.7, 'num_dense_layers_feasibility': 3, 'num_episodic_network_unit': 192, 'learning_rate': 0.005}, 'Experiment4': {'num_unit_gru': 80, 'num_layers_gru': 2, 'num_episodes': 2, 'num_dense_layer_feasibility_units': 48, 'dropout_rate': 0.6, 'num_dense_layers_feasibility': 1, 'num_episodic_network_unit': 128, 'learning_rate': 0.005}, 'Experiment5': {'num_unit_gru': 80, 'num_layers_gru': 2, 'num_episodes': 3, 'num_dense_layer_feasibility_units': 64, 'dropout_rate': 0.5, 'num_dense_layers_feasibility': 1, 'num_episodic_network_unit': 192, 'learning_rate': 0.005}}

Experiment_Dic

{'Experiment0': {'num_unit_gru': 64,
  'num_layers_gru': 2,
  'num_episodes': 3,
  'num_dense_layer_feasibility_units': 32,
  'dropout_rate': 0.6,
  'num_dense_layers_feasibility': 1,
  'num_episodic_network_unit': 192,
  'learning_rate': 0.001},
 'Experiment1': {'num_unit_gru': 80,
  'num_layers_gru': 2,
  'num_episodes': 2,
  'num_dense_layer_feasibility_units': 64,
  'dropout_rate': 0.7,
  'num_dense_layers_feasibility': 3,
  'num_episodic_network_unit': 192,
  'learning_rate': 0.005},
 'Experiment2': {'num_unit_gru': 100,
  'num_layers_gru': 2,
  'num_episodes': 3,
  'num_dense_layer_feasibility_units': 64,
  'dropout_rate': 0.5,
  'num_dense_layers_feasibility': 1,
  'num_episodic_network_unit': 64,
  'learning_rate': 0.005},
 'Experiment3': {'num_unit_gru': 80,
  'num_layers_gru': 2,
  'num_episodes': 2,
  'num_dense_layer_feasibility_units': 32,
  'dropout_rate': 0.7,
  'num_dense_layers_feasibility': 3,
  'num_episodic_network_unit': 192,
  'learning_rate': 0.005},
 'Experiment

In [25]:
def run_experiments(Experiment_Dic,
                    Experiment_No,
                    embedding_matrix,
                    ndim = 100,
                    tpu_enabled=0,
                    num_training_samples=1024,
                    num_validation_samples = 256,
                    num_epochs = 50,
                    batch_size = 10):
    num_training_samples = int((num_training_samples//128)*128)
    num_validation_samples = int((num_validation_samples//128)*128)
    #get the experiment details
    ExperimentNo = 'Experiment'+str(Experiment_No)
    num_unit_gru = Experiment_Dic[ExperimentNo]['num_unit_gru']
    num_layers_gru = Experiment_Dic[ExperimentNo]['num_layers_gru']
    num_episodes = Experiment_Dic[ExperimentNo]['num_episodes']
    num_dense_layer_feasibility_units = Experiment_Dic[ExperimentNo]['num_dense_layer_feasibility_units']
    dropout_rate = Experiment_Dic[ExperimentNo]['dropout_rate']
    num_dense_layers_feasibility = Experiment_Dic[ExperimentNo]['num_dense_layers_feasibility']
    num_episodic_network_unit = Experiment_Dic[ExperimentNo]['num_episodic_network_unit']
    learning_rate = Experiment_Dic[ExperimentNo]['learning_rate']
    
    
    if tpu_enabled==0:
        #When GPU ENABLED
        answer_model,\
        encoder_model,\
        decoder_model,\
        feasibility_model = create_models(
                                      embedding_matrix = embedding_matrix,
                                      max_context_length = context_maxlen,
                                      max_question_length = question_maxlen,
                                      max_answer_length = answer_maxlen,
                                      num_unit_gru = num_unit_gru,
                                      num_layers_gru = num_layers_gru,
                                      ndim =ndim,
                                      num_episodes = num_episodes,
                                      num_dense_layer_feasibility_units = num_dense_layer_feasibility_units,
                                      dropout_rate = dropout_rate,
                                      num_dense_layers_feasibility = num_dense_layers_feasibility,
                                      num_episodic_network_unit = num_episodic_network_unit)

        adam_optim = keras.optimizers.Adam(learning_rate=learning_rate)
        answer_model.compile(optimizer=adam_optim,
                                   loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                                   metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
                                )

        answer_model.summary()
        feasibility_model.compile(optimizer=adam_optim,
                                   loss=tf.keras.losses.BinaryCrossentropy(),
                                   metrics=[tf.keras.metrics.BinaryAccuracy()]
                                   )
        feasibility_model.summary()
    else: 
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.experimental.TPUStrategy(resolver)
        batch_size = 128*8
        with strategy.scope():
            answer_model,\
            encoder_model,\
            decoder_model,\
            feasibility_model = create_models(
                                          embedding_matrix = embedding_matrix,
                                          max_context_length = context_maxlen,
                                          max_question_length = question_maxlen,
                                          max_answer_length = answer_maxlen,
                                          num_unit_gru = num_unit_gru,
                                          num_layers_gru = num_layers_gru,
                                          ndim =ndim,
                                          num_episodes = num_episodes,
                                          num_dense_layer_feasibility_units = num_dense_layer_feasibility_units,
                                          dropout_rate = dropout_rate,
                                          num_dense_layers_feasibility = num_dense_layers_feasibility,
                                          num_episodic_network_unit = num_episodic_network_unit)

            adam_optim = keras.optimizers.Adam(learning_rate=learning_rate)
            answer_model.compile(optimizer=adam_optim,
                                       loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                                       metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
                                )

            answer_model.summary()
            feasibility_model.compile(optimizer=adam_optim,
                                       loss=tf.keras.losses.BinaryCrossentropy(),
                                       metrics=[tf.keras.metrics.BinaryAccuracy()]
                                       )
            feasibility_model.summary()
        
        
        
    history_answer_model = answer_model.fit(
                                         {'Context_Input':train_context_padded_seq[:num_training_samples],
                                         'Question_Input':train_question_seq_padded[:num_training_samples],
                                         'Answer_Input':train_answer_input_seq_padded[:num_training_samples] },
                                         {'Answer_output':train_answer_target_seq_padded[:num_training_samples] },
                                     epochs=num_epochs,
                                     batch_size=batch_size,
                                     validation_data=([val_context_padded_seq[:num_validation_samples],
                                                       val_question_seq_padded[:num_validation_samples],
                                                       val_answer_input_seq_padded[:num_validation_samples]],
                                                       val_answer_target_seq_padded[:num_validation_samples]))
    answer_model.save(ExperimentNo+'original_model_answer_model.h5')
    encoder_model.save(ExperimentNo+'original_model_encoder_model.h5')
    decoder_model.save(ExperimentNo+'original_model_decoder_model.h5')
    with open(ExperimentNo+'original_model_'+'history_answer_model', 'wb') as file_history:
        pickle.dump(history_answer_model.history, file_history)
    _,encoder_prediction,_ = encoder_model.predict([train_context_padded_seq[:num_training_samples],
                                                    train_question_seq_padded[:num_training_samples]])
    _,encoder_validation_prediction,_ = encoder_model.predict([val_context_padded_seq[:num_validation_samples],
                                                               val_question_seq_padded[:num_validation_samples]])
    history_feasibility_model = feasibility_model.fit(encoder_prediction,
                                                      train_answer_impossible[:num_training_samples],
                                                      epochs=num_epochs,
                                                      batch_size=batch_size,
                                                      validation_data = 
                                                            (encoder_validation_prediction,
                                                             val_answer_impossible[:num_validation_samples])
                                                      )



    feasibility_model.save(ExperimentNo+'original_model_feasibility_model.h5')
    with open(ExperimentNo+'original_model_'+'history_feasibility_model', 'wb') as file_history:
        pickle.dump(history_feasibility_model.history, file_history)
    

In [26]:
def create_inference_model(Experiment_Dic,
                           Experiment_No,
                           embedding_matrix,
                           ndim = 100):
    ExperimentNo = 'Experiment'+str(Experiment_No)
    num_unit_gru = Experiment_Dic[ExperimentNo]['num_unit_gru']
    num_layers_gru = Experiment_Dic[ExperimentNo]['num_layers_gru']
    num_episodes = Experiment_Dic[ExperimentNo]['num_episodes']
    num_dense_layer_feasibility_units = Experiment_Dic[ExperimentNo]['num_dense_layer_feasibility_units']
    dropout_rate = Experiment_Dic[ExperimentNo]['dropout_rate']
    num_dense_layers_feasibility = Experiment_Dic[ExperimentNo]['num_dense_layers_feasibility']
    num_episodic_network_unit = Experiment_Dic[ExperimentNo]['num_episodic_network_unit']
    learning_rate = Experiment_Dic[ExperimentNo]['learning_rate']
    
    inference_answer_model,\
    inference_encoder_model,\
    inference_decoder_model,\
    inference_feasibility_model = create_models(
                                          embedding_matrix = embedding_matrix,
                                          max_context_length = context_maxlen,
                                          max_question_length = question_maxlen,
                                          max_answer_length = answer_maxlen,
                                          num_unit_gru = num_unit_gru,
                                          num_layers_gru = num_layers_gru,
                                          ndim =ndim,
                                          num_episodes = num_episodes,
                                          num_dense_layer_feasibility_units = num_dense_layer_feasibility_units,
                                          dropout_rate = dropout_rate,
                                          num_dense_layers_feasibility = num_dense_layers_feasibility,
                                          num_episodic_network_unit = num_episodic_network_unit)


    inference_answer_model.load_weights(ExperimentNo+'original_model_answer_model.h5')
    inference_encoder_model.load_weights(ExperimentNo+'original_model_encoder_model.h5')
    inference_decoder_model.load_weights(ExperimentNo+'original_model_decoder_model.h5')
    inference_feasibility_model.load_weights(ExperimentNo+'original_model_feasibility_model.h5')
    return (inference_answer_model,inference_encoder_model,inference_decoder_model,inference_feasibility_model)

In [27]:
#Run Experiments
#change the num_training_samples and num_validation_samples make sure its multiple of 128 when running on tpu
#change to tpu_enabled = 1 when running on tpu 
#Change the batch_size and epochs too
run_experiments(Experiment_Dic=Experiment_Dic,
                Experiment_No=0,
                embedding_matrix=embedding_matrix,
                ndim = 100,
                tpu_enabled=0,
                num_training_samples=1024,
                num_validation_samples = 256,
                num_epochs = 50,
                batch_size = 10)

Context output shape (None, None, 64)
Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Question_Input (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
Question_Embedding (Embedding)  (None, None, 100)    8870200     Question_Input[0][0]             
__________________________________________________________________________________________________
Question_Bid_Layer0 (Bidirectio (None, None, 64)     63744       Question_Embedding[0][0]         
__________________________________________________________________________________________________
batch_normalization_23 (BatchNo (None, None, 64)     256         Question_Bid_Layer0[0][0]        
_____________________________________________________

1024/1024 [==============================] - 156s 152ms/sample - loss: 3.7945 - sparse_categorical_accuracy: 0.1045 - val_loss: 3.5886 - val_sparse_categorical_accuracy: 0.1304
Epoch 2/50
1024/1024 [==============================] - 135s 132ms/sample - loss: 2.9195 - sparse_categorical_accuracy: 0.2477 - val_loss: 3.2220 - val_sparse_categorical_accuracy: 0.2014
Epoch 3/50
1024/1024 [==============================] - 134s 130ms/sample - loss: 2.0790 - sparse_categorical_accuracy: 0.3681 - val_loss: 2.2529 - val_sparse_categorical_accuracy: 0.4115
Epoch 4/50
1024/1024 [==============================] - 134s 131ms/sample - loss: 1.5904 - sparse_categorical_accuracy: 0.4274 - val_loss: 2.0181 - val_sparse_categorical_accuracy: 0.4348
Epoch 5/50
1024/1024 [==============================] - 125s 122ms/sample - loss: 1.4860 - sparse_categorical_accuracy: 0.4354 - val_loss: 2.0688 - val_sparse_categorical_accuracy: 0.4446
Epoch 6/50
1024/1024 [==============================] - 124s 121ms/samp

1024/1024 [==============================] - 137s 134ms/sample - loss: 1.0882 - sparse_categorical_accuracy: 0.4741 - val_loss: 2.0757 - val_sparse_categorical_accuracy: 0.4815
Epoch 45/50
1024/1024 [==============================] - 123s 120ms/sample - loss: 1.0777 - sparse_categorical_accuracy: 0.4726 - val_loss: 2.0803 - val_sparse_categorical_accuracy: 0.4815
Epoch 46/50
1024/1024 [==============================] - 127s 124ms/sample - loss: 1.0748 - sparse_categorical_accuracy: 0.4753 - val_loss: 2.0838 - val_sparse_categorical_accuracy: 0.4776
Epoch 47/50
1024/1024 [==============================] - 125s 122ms/sample - loss: 1.0655 - sparse_categorical_accuracy: 0.4743 - val_loss: 2.0841 - val_sparse_categorical_accuracy: 0.4796
Epoch 48/50
1024/1024 [==============================] - 128s 125ms/sample - loss: 1.0522 - sparse_categorical_accuracy: 0.4769 - val_loss: 2.0877 - val_sparse_categorical_accuracy: 0.4786
Epoch 49/50
1024/1024 [==============================] - 123s 120ms

Epoch 43/50
1024/1024 [==============================] - 0s 164us/sample - loss: 0.5819 - binary_accuracy: 0.6904 - val_loss: 0.7350 - val_binary_accuracy: 0.5820
Epoch 44/50
1024/1024 [==============================] - 0s 162us/sample - loss: 0.5883 - binary_accuracy: 0.6914 - val_loss: 0.7296 - val_binary_accuracy: 0.6016
Epoch 45/50
1024/1024 [==============================] - 0s 162us/sample - loss: 0.5872 - binary_accuracy: 0.6973 - val_loss: 0.7454 - val_binary_accuracy: 0.5898
Epoch 46/50
1024/1024 [==============================] - 0s 161us/sample - loss: 0.5861 - binary_accuracy: 0.6885 - val_loss: 0.7129 - val_binary_accuracy: 0.6055
Epoch 47/50
1024/1024 [==============================] - 0s 162us/sample - loss: 0.5857 - binary_accuracy: 0.6865 - val_loss: 0.7153 - val_binary_accuracy: 0.5938
Epoch 48/50
1024/1024 [==============================] - 0s 162us/sample - loss: 0.5831 - binary_accuracy: 0.7021 - val_loss: 0.7271 - val_binary_accuracy: 0.5977
Epoch 49/50
1024/1024 

In [ ]:
#Run Inference
inference_answer_model,\
inference_encoder_model,\
inference_decoder_model,\
inference_feasibility_model = create_inference_model(Experiment_Dic=Experiment_Dic,
                                                     Experiment_No=0,
                                                     embedding_matrix=embedding_matrix,
                                                     ndim = 100)

In [33]:
for seq_index in range(3000,3010):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    context_input_seq = train_context_padded_seq[seq_index: seq_index+ 1]
    question_input_seq = train_question_seq_padded[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(context_input_seq,
                                       question_input_seq,
                                       inference_encoder_model,
                                       inference_decoder_model)
    print("question:",' '.join([id_vocab.get(i) for i in train_question_seq_padded[seq_index].tolist() if i !=0]))
    print('Predicted Answer:', decoded_sentence)
    act_answer = ' '.join([id_vocab.get(i) for i in train_answer_input_seq_padded[seq_index].tolist() if i !=0])
    print('Actual answer:',act_answer)

question: what train line connects florida to the north
Predicted Answer: </s> 
Actual answer: <s> amtrak operating numerous lines throughout connecting the state s largest cities
question: who jointly owns sportsnet and tsn
Predicted Answer: </s> 
Actual answer: <s> </s>
question: who revealed the starting points of this evolution to be economic deterioration and political confusion
Predicted Answer: </s> 
Actual answer: <s> ervin staub </s>
question: what is yeovilton home to
Predicted Answer: </s> 
Actual answer: <s> the royal naval air station in yeovilton is one of britain
question: what are two smaller galleries in nanjing
Predicted Answer: </s> 
Actual answer: <s> red chamber art garden and jinling stone gallery </s>
question: where did shuhda work
Predicted Answer: </s> 
Actual answer: <s> </s>
question: in what year did apple face multiple intellectual property lawsuits
Predicted Answer: </s> 
Actual answer: <s> 2005 </s>
question: how many people lived in kaliningrad in 1946
